<a href="https://colab.research.google.com/github/astromad/MyDeepLearningRepo/blob/master/PowerBallFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from pandas import read_csv
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import LSTM
from keras.layers import Dense, Dropout, Activation
from sklearn.preprocessing import StandardScaler

In [ ]:
window =10

In [ ]:
#powerball
url='https://www.texaslottery.com/export/sites/lottery/Games/Powerball/Winning_Numbers/powerball.csv'
df = read_csv(url, usecols= [4,5,6,7,8,9], header=None)
df.tail(20)

,4,5,6,7,8,9
1531,27,62,16,59,63,23
1532,1,12,66,20,33,21
1533,50,10,36,22,12,4
1534,47,63,1,46,7,7
1535,30,19,46,37,44,22
1536,27,12,43,47,26,5
1537,63,9,35,64,54,1
1538,47,60,57,65,54,19
1539,67,34,46,55,16,14
1540,24,52,40,22,64,10


In [ ]:
tdf=df.iloc[:-window,:]
tdf.tail(10)

,4,5,6,7,8,9
1531,27,62,16,59,63,23
1532,1,12,66,20,33,21
1533,50,10,36,22,12,4
1534,47,63,1,46,7,7
1535,30,19,46,37,44,22
1536,27,12,43,47,26,5
1537,63,9,35,64,54,1
1538,47,60,57,65,54,19
1539,67,34,46,55,16,14
1540,24,52,40,22,64,10


In [ ]:
scalar = StandardScaler().fit(tdf.values)
lotto_data = scalar.transform(tdf.values)
lotto_df = pd.DataFrame(data=lotto_data, index=tdf.index)

In [ ]:
rows = tdf.values.shape[0]
features = tdf.values.shape[1]
print('number of past winners',rows)
print('how many numbers to pick',features)

number of past winners 1541
how many numbers to pick 6


In [ ]:
train = np.empty([rows - window, window, features],dtype = float)
label = np.empty([rows-window,features],dtype = float)
for i in range(0,rows-window):
  train[i] = lotto_df.iloc[i:i+window, 0:features]
  label[i] = lotto_df.iloc[i+window:i+window+1,0:features]

In [ ]:
print(label[0])

[ 0.40617242  0.22906099 -0.85962025  0.86112689 -0.66239334 -1.58923769]


In [ ]:
batch_size = 32
model = Sequential()
model.add(LSTM(300,activation='tanh',input_shape=(window,features),return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(170,activation='tanh',return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(150,activation='tanh',return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(130,activation='tanh',return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(100,activation='tanh',return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(70,activation='tanh',return_sequences=False))
model.add(Dropout(0.1))
model.add(Dense(features))
model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 10, 300)           368400    
                                                                 
 dropout_6 (Dropout)         (None, 10, 300)           0         
                                                                 
 lstm_7 (LSTM)               (None, 10, 170)           320280    
                                                                 
 dropout_7 (Dropout)         (None, 10, 170)           0         
                                                                 
 lstm_8 (LSTM)               (None, 10, 150)           192600    
                                                                 
 dropout_8 (Dropout)         (None, 10, 150)           0         
                                                                 
 lstm_9 (LSTM)               (None, 10, 130)          

In [ ]:
model.fit(train,label,batch_size=batch_size,epochs=100)

Epoch 1/100
48/48 [==============================] - 13s 56ms/step - loss: 0.9980 - accuracy: 0.2084
Epoch 2/100
48/48 [==============================] - 3s 56ms/step - loss: 0.9926 - accuracy: 0.2260
Epoch 3/100
48/48 [==============================] - 3s 55ms/step - loss: 0.9900 - accuracy: 0.2266
Epoch 4/100
48/48 [==============================] - 3s 56ms/step - loss: 0.9884 - accuracy: 0.2201
Epoch 5/100
48/48 [==============================] - 3s 60ms/step - loss: 0.9872 - accuracy: 0.2319
Epoch 6/100
48/48 [==============================] - 3s 56ms/step - loss: 0.9869 - accuracy: 0.2391
Epoch 7/100
48/48 [==============================] - 3s 55ms/step - loss: 0.9840 - accuracy: 0.2195
Epoch 8/100
48/48 [==============================] - 3s 55ms/step - loss: 0.9868 - accuracy: 0.2338
Epoch 9/100
48/48 [==============================] - 3s 56ms/step - loss: 0.9821 - accuracy: 0.2273
Epoch 10/100
48/48 [==============================] - 3s 60ms/step - loss: 0.9836 - accuracy: 0.237

In [ ]:
resultDict = {}
for i in range(window,0,-1):
    to_predict= scalar.transform(df.tail(i).values)
    output = model.predict(np.array([to_predict]))
    resultDict[i]=scalar.inverse_transform(output).astype(int)[0]

1/1 [==============================] - 0s 26ms/step


In [ ]:
print("PowerBall Prediction")
[print(values) for values in resultDict.values()]

PowerBall Prediction
[47 11 19 11 18 16]
[45 13 36 14 20  7]
[13 28 30 23 12 19]
[21 41 47 15 20 19]
[18 38 54 18 41 11]
[20 41 44 24 35 12]
[30 35 45 25 26 11]
[26 44 49 20 42 11]
[24 44 32 41 37 12]
[34 36 31 33 36 14]


[None, None, None, None, None, None, None, None, None, None]